In [1]:
import numpy as np
import torch
import torch.optim as optim
import pickle
import pandas as pd
import sys
import copy

sys.path.append("..\\..\\src")

from datasets import CV2ImageDataset, dataset_loader
from tf_model import Net
#from cifar_model import Net
from tqdm import tqdm
from model_class import NeuralNet
from run_phase import run_phase
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
import pickle

from meta_learning_tools import MetaLearn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
def model_prototype():
    net = Net()
    nn_model = NeuralNet(net)
    net = nn_model.get_model()
    return net

In [3]:
with open('saved_cifar_model_weights_dictionary_with_tl.pkl', 'rb') as fp:
    model_weight_dict = pickle.load(fp)

In [4]:
model_list = []
for key in tqdm(model_weight_dict.keys()):
    model_weight = model_weight_dict[key]
    model = model_prototype()
    model.load_state_dict(model_weight)
    model.cpu()
    model.eval()
    model_list.append(model)

100%|████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.74s/it]


In [13]:
train_df = pd.read_csv('data\\train.csv')
val_df = pd.read_csv('data\\test.csv')



print('train_df initial shape: ',train_df.shape)
print('val_df intial shape: ', val_df.shape)

train_df = train_df[0:10000]
val_df = val_df[0:7500]


print('train_df shape: ',train_df.shape)
print('val_df shape: ', val_df.shape)

train_df initial shape:  (50000, 5)
val_df intial shape:  (10000, 5)
train_df shape:  (10000, 5)
val_df shape:  (7500, 5)


In [ ]:
meta_learn = MetaLearn(model_list)
tta_transforms = [
    A.augmentations.transforms.ChannelShuffle(p=0.5),
    A.HorizontalFlip(p=0.5), 
]

print('Generate Dictionary of Metalearning Dataframes')
train_tta_dict= meta_learn.generate_tta_dict_for_folds(train_df , tta_transforms, device)
val_tta_dict= meta_learn.generate_tta_dict_for_folds(val_df , tta_transforms, device)

Generate Dictionary of Metalearning Dataframes


  0%|                                                                | 0/10000 [00:00<?, ?it/s]E:\anaconda3\envs\pytorch\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 37%|███████████████████▍                                 | 3675/10000 [11:52<19:55,  5.29it/s]

In [ ]:
with open('train_tta_dict.pkl', 'wb') as fp:
    pickle.dump(train_tta_dict, fp)
with open('val_tta_dict.pkl', 'wb') as fp:
    pickle.dump(val_tta_dict, fp)

In [ ]:
print(val_tta_dict.keys())#, .keys())

In [ ]:
from datasets import CV2ImageDataset, dataset_loader
from run_phase import run_phase
batch_size = 64
aug = A.Compose([ 
    A.Normalize(),            
    ToTensorV2()])
val_ds = CV2ImageDataset(val_df, transform=aug, device = device)
val_ds_l = dataset_loader(val_ds, batch_size = batch_size)
valloader = val_ds_l.get_dataloader()
# Just Sanity Check
for i, model in enumerate(model_list):
    print('model : ', i)
    val_phase = run_phase(valloader, model.to(device),'Validation', device, loss = torch.nn.CrossEntropyLoss( ))
    val_accuracy_meter, val_loss_meter =  val_phase.run()
    model.cpu()
    model.eval()
    

In [ ]:
train_tta_df=meta_learn.create_meta_learn_labels_on_dict(train_tta_dict)
val_tta_df=meta_learn.create_meta_learn_labels_on_dict(val_tta_dict)
print( val_tta_df.shape,train_tta_df.shape)

In [ ]:
val_tta_df.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

val_acc, val_log_loss = meta_learn.do_meta_learning_on_tta_dicts(RandomForestClassifier(), train_tta_df, val_tta_df, 'custom')

print(val_acc, val_log_loss)